<a href="https://colab.research.google.com/github/RulCab/ChallengeSolvento/blob/main/challenge_solvento_ra%C3%BAl_cabrera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports
from enum import Enum
from collections import defaultdict


In [2]:
#definición de las clases

class Item:
    def __init__(self, item_id, name, weight, price):
        self.item_id = item_id
        self.name = name
        self.weight = weight
        self.price = price

class Purchase:
    def __init__(self, purchase_id, customer_name, items, date, address, state='Pending'):
        self.purchase_id = purchase_id
        self.customer_name = customer_name  # nombre del cliente
        self.items = items  # lista de tuples (Item, quantity)
        self.date = date  # fecha de la compra
        self.address = address  # instancia de la clase Address
        self.state = state
        self.delivery_date = None  # fecha de entrega planificada, inicialmente None

    @property
    def total_weight(self):
        return sum(item.weight * quantity for item, quantity in self.items)

    @property
    def total_price(self):
        return sum(item.price * quantity for item, quantity in self.items)

    def schedule_delivery(self, delivery_date):
        self.delivery_date = delivery_date

class Truck:
    def __init__(self, plate_number, max_weight, address, working_days, available=True):
        self.plate_number = plate_number
        self.max_weight = max_weight
        self.address = address
        self.available = available
        self.working_days = set(working_days)  # días de la semana en los que el camión está disponible

    def is_available(self, day):
        # comprobar si el camión está disponible en un día específico
        return day in self.working_days

class TripState(Enum):
    PLANNED = "Planned"
    IN_PROGRESS = "In Progress"
    COMPLETED = "Completed"
    CANCELLED = "Cancelled"

class Trip:
    def __init__(self, trip_id, zip_codes, trucks=None, departure=None):
        self.trip_id = trip_id
        self.zip_codes = zip_codes  # lista de códigos postales
        self.trucks = trucks or []  # lista de camiones asignados
        self.departure = departure  # fecha/hora de salida
        self.state = TripState.PLANNED  # estado inicial del viaje

    def add_truck(self, truck):
        if len(self.trucks) < 3:
            self.trucks.append(truck)
        else:
            raise ValueError("No se pueden asignar más de 3 camiones por viaje.")

    def update_state(self, new_state):
        if new_state in TripState:
            self.state = new_state
        else:
            raise ValueError("Estado no válido para el viaje.")

class Address:
    def __init__(self, street, city, zip_code, country):
        self.street = street
        self.city = city
        self.zip_code = zip_code
        self.country = country


In [3]:
# Simulando tablas de una base de datos
db = {
    "items": [
        {"item_id": 1, "name": "Navigation Log Pose", "weight": 0.2, "price": 300.00},
        {"item_id": 2, "name": "Den Den Mushi", "weight": 1.0, "price": 150.00},
        {"item_id": 3, "name": "Tony Chopper Hat", "weight": 0.3, "price": 15.00},
        {"item_id": 4, "name": "Roronoa Zoro's Swords Set", "weight": 5.0, "price": 500.00},
        {"item_id": 5, "name": "Straw Hat", "weight": 0.2, "price": 20.00},
        {"item_id": 6, "name": "Devil Fruit", "weight": 0.5, "price": 1000.00}
    ],
    "purchases": [
        {"purchase_id": 1, "customer_name": "Tony Chopper", "items": [(3, 2)], "date": "2023-10-01", "address": "Doctor's Lab, Drum Island", "state": "Pending"},
        {"purchase_id": 2, "customer_name": "Monkey D. Luffy", "items": [(5, 1), (6, 1)], "date": "2023-10-05", "address": "Thousand Sunny, Grand Line", "state": "OnTrip"},
        {"purchase_id": 3, "customer_name": "Nico Robin", "items": [(6, 1)], "date": "2023-10-06", "address": "Archaeological Camp, Ohara", "state": "Delivered"}
    ],
    "trucks": [
        {"plate_number": "MERRY GO", "max_weight": 5000, "address": "Dock #0, East Blue", "working_days": {"Monday", "Wednesday", "Friday"}, "available": True},
        {"plate_number": "GOING MERRY", "max_weight": 10000, "address": "Dock #1, East Blue", "working_days": {"Monday", "Tuesday", "Thursday"}, "available": True},
        {"plate_number": "THOUSAND SUNNY", "max_weight": 12000, "address": "Dock #2, Grand Line", "working_days": {"Wednesday", "Friday"}, "available": True}
    ],
    "trips": []
}

# obtener todos los ítems disponibles en la base de datos
def get_all_items():
    # lista de todos los ítems.
    return db["items"]

# obtener los ítems de una compra específica basada en su ID
def get_purchase_items(purchase_id):
    # busca la compra con el ID especificado y devuelve los ítems correspondientes
    purchase = next((p for p in db["purchases"] if p["purchase_id"] == purchase_id), None)
    if purchase:
        # compila una lista de ítems y cantidades compradas de la compra encontrada
        return [(next((item for item in db["items"] if item["item_id"] == item_id), None), quantity) for item_id, quantity in purchase["items"]]
    return []  # si no se encuentra la compra, devuelve una lista vacía

# agregar un ítem adicional a una compra existente
def add_item_to_purchase(purchase_id, item_id, quantity):
    # busca la compra por su ID y añade el ítem con la cantidad especificada
    purchase = next((p for p in db["purchases"] if p["purchase_id"] == purchase_id), None)
    if purchase:
        # añade el ítem a la lista de ítems de la compra
        purchase["items"].append((item_id, quantity))

# prueba de la función
print(get_all_items())

print(get_purchase_items(1))  # Debería mostrar los ítems comprados por Tony Chopper



[{'item_id': 1, 'name': 'Navigation Log Pose', 'weight': 0.2, 'price': 300.0}, {'item_id': 2, 'name': 'Den Den Mushi', 'weight': 1.0, 'price': 150.0}, {'item_id': 3, 'name': 'Tony Chopper Hat', 'weight': 0.3, 'price': 15.0}, {'item_id': 4, 'name': "Roronoa Zoro's Swords Set", 'weight': 5.0, 'price': 500.0}, {'item_id': 5, 'name': 'Straw Hat', 'weight': 0.2, 'price': 20.0}, {'item_id': 6, 'name': 'Devil Fruit', 'weight': 0.5, 'price': 1000.0}]
[({'item_id': 3, 'name': 'Tony Chopper Hat', 'weight': 0.3, 'price': 15.0}, 2)]


In [4]:
#Funciones para programar viajes y asignar camiones

# asignar camiones a viajes basándose en la capacidad de carga y disponibilidad
def assign_trucks_to_trips(trips, trucks, items_db):
    for trip in trips:
        # calcula el peso total de los ítems en el viaje.
        trip_weight = sum(items_db[item_id - 1]['weight'] * quantity for purchase in trip['purchases'] for item_id, quantity in purchase['items'])
        # busca un camión disponible que pueda manejar el peso total
        suitable_truck = None
        for truck in trucks:
            if truck['max_weight'] >= trip_weight and truck['available']:
                suitable_truck = truck  # asigna el camión si cumple con los requisitos
                break  # sale del bucle después de encontrar un camión adecuado
        if suitable_truck:
            # si se encontró un camión adecuado, asigna el camión al viaje
            trip['assigned_truck'] = suitable_truck['plate_number']
            suitable_truck['available'] = False  # marca el camión como no disponible
        else:
            # si no se encuentra un camión disponible, error
            trip['assigned_truck'] = 'No available truck'
            print(f"Warning: No truck available for trip {trip['zip_codes']} with required capacity {trip_weight}.")

# planificar viajes basándose en los códigos postales y el número máximo de paradas
def plan_trips(purchases, max_stops=3):
    postal_groups = defaultdict(list)  # agrupa las compras por dirección postal
    for purchase in purchases:
        # agrega la compra al grupo de su código postal correspondiente
        postal_groups[purchase['address']].append(purchase)
    trips = []  # lista para almacenar los viajes planificados
    for postal, purchase_list in postal_groups.items():
        # agrupar hasta max_stops códigos postales
        trip_postals = [postal] + [p for p in postal_groups if p != postal][:max_stops-1]
        # compila las compras para esos códigos postales
        trip_purchases = [p for plist in [postal_groups[p] for p in trip_postals] for p in plist]
        # crea y añade el viaje planificado a la lista de viajes
        trips.append({'zip_codes': trip_postals, 'purchases': trip_purchases})
    return trips  # devuelve la lista de viajes planificados

items_list = db["items"]
purchases_list = db["purchases"]
trucks_list = db["trucks"]

trips = plan_trips(purchases_list)
assign_trucks_to_trips(trips, trucks_list, items_list)

for trip in trips:
    print(trip)




{'zip_codes': ["Doctor's Lab, Drum Island", 'Thousand Sunny, Grand Line', 'Archaeological Camp, Ohara'], 'purchases': [{'purchase_id': 1, 'customer_name': 'Tony Chopper', 'items': [(3, 2)], 'date': '2023-10-01', 'address': "Doctor's Lab, Drum Island", 'state': 'Pending'}, {'purchase_id': 2, 'customer_name': 'Monkey D. Luffy', 'items': [(5, 1), (6, 1)], 'date': '2023-10-05', 'address': 'Thousand Sunny, Grand Line', 'state': 'OnTrip'}, {'purchase_id': 3, 'customer_name': 'Nico Robin', 'items': [(6, 1)], 'date': '2023-10-06', 'address': 'Archaeological Camp, Ohara', 'state': 'Delivered'}], 'assigned_truck': 'MERRY GO'}
{'zip_codes': ['Thousand Sunny, Grand Line', "Doctor's Lab, Drum Island", 'Archaeological Camp, Ohara'], 'purchases': [{'purchase_id': 2, 'customer_name': 'Monkey D. Luffy', 'items': [(5, 1), (6, 1)], 'date': '2023-10-05', 'address': 'Thousand Sunny, Grand Line', 'state': 'OnTrip'}, {'purchase_id': 1, 'customer_name': 'Tony Chopper', 'items': [(3, 2)], 'date': '2023-10-01',

In [5]:
#visualización de datos
import pandas as pd

# crear DataFrames para cada componente de la base de datos
items_df = pd.DataFrame(db['items'])
purchases_df = pd.DataFrame(db['purchases'])
trucks_df = pd.DataFrame(db['trucks'])
trips_df = pd.DataFrame(trips)

# mostrar los DataFrames
print("Items:")
display(items_df)
print("Purchases:")
display(purchases_df)
print("Trucks:")
display(trucks_df)
print("Trips:")
display(trips_df)


Items:


,item_id,name,weight,price
0,1,Navigation Log Pose,0.2,300.0
1,2,Den Den Mushi,1.0,150.0
2,3,Tony Chopper Hat,0.3,15.0
3,4,Roronoa Zoro's Swords Set,5.0,500.0
4,5,Straw Hat,0.2,20.0
5,6,Devil Fruit,0.5,1000.0


Purchases:


,purchase_id,customer_name,items,date,address,state
0,1,Tony Chopper,"[(3, 2)]",2023-10-01,"Doctor's Lab, Drum Island",Pending
1,2,Monkey D. Luffy,"[(5, 1), (6, 1)]",2023-10-05,"Thousand Sunny, Grand Line",OnTrip
2,3,Nico Robin,"[(6, 1)]",2023-10-06,"Archaeological Camp, Ohara",Delivered


Trucks:


,plate_number,max_weight,address,working_days,available
0,MERRY GO,5000,"Dock #0, East Blue","{Friday, Monday, Wednesday}",False
1,GOING MERRY,10000,"Dock #1, East Blue","{Tuesday, Monday, Thursday}",False
2,THOUSAND SUNNY,12000,"Dock #2, Grand Line","{Friday, Wednesday}",False


Trips:


,zip_codes,purchases,assigned_truck
0,"[Doctor's Lab, Drum Island, Thousand Sunny, Gr...","[{'purchase_id': 1, 'customer_name': 'Tony Cho...",MERRY GO
1,"[Thousand Sunny, Grand Line, Doctor's Lab, Dru...","[{'purchase_id': 2, 'customer_name': 'Monkey D...",GOING MERRY
2,"[Archaeological Camp, Ohara, Doctor's Lab, Dru...","[{'purchase_id': 3, 'customer_name': 'Nico Rob...",THOUSAND SUNNY


In [6]:
import ipywidgets as widgets
from IPython.display import display

# widget para seleccionar un viaje y mostrar detalles
trip_selector = widgets.Dropdown(
    options=[(f"Trip to {', '.join(t['zip_codes'])}", idx) for idx, t in enumerate(trips)],
    description='Select Trip:'
)

def show_trip_details(change):
    idx = change['new']
    selected_trip = trips[idx]
    print(f"Selected Trip to {', '.join(selected_trip['zip_codes'])}")
    print(f"Assigned Truck: {selected_trip['assigned_truck']}")
    # convertir lista de compras a DataFrame para mejor visualización
    trip_purchases_df = pd.DataFrame(selected_trip['purchases'])
    display(trip_purchases_df)

trip_selector.observe(show_trip_details, names='value')
display(trip_selector)


Dropdown(description='Select Trip:', options=(("Trip to Doctor's Lab, Drum Island, Thousand Sunny, Grand Line,…

Selected Trip to Thousand Sunny, Grand Line, Doctor's Lab, Drum Island, Archaeological Camp, Ohara
Assigned Truck: GOING MERRY


,purchase_id,customer_name,items,date,address,state
0,2,Monkey D. Luffy,"[(5, 1), (6, 1)]",2023-10-05,"Thousand Sunny, Grand Line",OnTrip
1,1,Tony Chopper,"[(3, 2)]",2023-10-01,"Doctor's Lab, Drum Island",Pending
2,3,Nico Robin,"[(6, 1)]",2023-10-06,"Archaeological Camp, Ohara",Delivered


Selected Trip to Archaeological Camp, Ohara, Doctor's Lab, Drum Island, Thousand Sunny, Grand Line
Assigned Truck: THOUSAND SUNNY


,purchase_id,customer_name,items,date,address,state
0,3,Nico Robin,"[(6, 1)]",2023-10-06,"Archaeological Camp, Ohara",Delivered
1,1,Tony Chopper,"[(3, 2)]",2023-10-01,"Doctor's Lab, Drum Island",Pending
2,2,Monkey D. Luffy,"[(5, 1), (6, 1)]",2023-10-05,"Thousand Sunny, Grand Line",OnTrip


Selected Trip to Doctor's Lab, Drum Island, Thousand Sunny, Grand Line, Archaeological Camp, Ohara
Assigned Truck: MERRY GO


,purchase_id,customer_name,items,date,address,state
0,1,Tony Chopper,"[(3, 2)]",2023-10-01,"Doctor's Lab, Drum Island",Pending
1,2,Monkey D. Luffy,"[(5, 1), (6, 1)]",2023-10-05,"Thousand Sunny, Grand Line",OnTrip
2,3,Nico Robin,"[(6, 1)]",2023-10-06,"Archaeological Camp, Ohara",Delivered
